In [1]:
import pandas as pd
import xgboost as xgb
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter # For logging values within training loop
import torch.onnx

In [2]:
# Load dataset
df = pd.read_csv('AmazonDataSales_v2.csv', low_memory=False)
# Drop all columns except 'amount', 'category', 'size', 'quantity'
df = df[['amount', 'category', 'size', 'qty']]

# One-hot encode the 'category', 'size', and 'qty' columns
# Select all columns except 'amount' as feature columns
feature_columns = df.columns.drop('amount')
# One-hot encode the feature columns
df_encoded = pd.get_dummies(df, columns=feature_columns)
df_encoded.columns

Index(['amount', 'category_blouse', 'category_bottom', 'category_dupatta',
       'category_ethnic dress', 'category_kurta', 'category_saree',
       'category_set', 'category_top', 'category_western dress', 'size_3xl',
       'size_4xl', 'size_5xl', 'size_6xl', 'size_free', 'size_l', 'size_m',
       'size_s', 'size_xl', 'size_xs', 'size_xxl', 'qty_0', 'qty_1', 'qty_2',
       'qty_3'],
      dtype='object')

In [3]:
#Dropping the one-hot columns XS, 6xl, 4xl, 5xl, free, ethnic dress, bottom, saree, blouse, dupatta, all qty except 1 would help the model?   
df_encoded.drop(['size_xs', 'size_4xl', 'size_5xl', 'size_6xl', 'size_free', 'qty_0', 'qty_2',
       'qty_3', 'category_blouse', 'category_dupatta','category_ethnic dress', 'category_saree', 'category_bottom',], axis=1, inplace=True)

In [4]:
# Assuming 'df' contains your dataset
X = df_encoded.drop('amount', axis=1)  # Features
y = df['amount']  # Target

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to train and evaluate a model
def train_evaluate_model(model, X_train, y_train, X_test, y_test):
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Calculate r2 score and rsme
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    
    return r2, rmse

#define model
regressor = xgb.XGBRegressor(objective='reg:squarederror')
#train model
regressor_train = train_evaluate_model(regressor, X_train, y_train, X_test, y_test)
print(f"R2/MSE:{regressor_train}")


R2/MSE:(0.39710457938755983, 191.61742156422437)


Conclusion: Removing the low frequency one-hot feature columns did not help the R2 value